# SH_mu_nu_reco_performance
This notebook analyzes muon and neutrino reconstruction performance for Solid Hydrogen events using ROOT (PyROOT) and RDataFrame.

It loads a ROOT file (`Rough_tree`), applies filters based on reconstruction quality (reco_flag), event topology (true_flag), and track multiplicity (track_flag), then produces comprehensive histograms and plots showing:
- **Muon properties**: momentum components (x,y,z), total momentum, and energy residuals
- **Reconstruction performance**: comparison of true vs reconstructed quantities
- **Neutrino residuals**: energy and momentum components derived from muon properties and beam direction
- **Resolution studies**: momentum resolution as a function of number of points and total momentum for single-track events
- **2D correlation plots**: reco vs true momentum components

Results are saved to `Plots/SH_mu_nu_reco_performance/SH_reco_performance.root` with detailed histograms and PDF plots.

Dependencies: ROOT (PyROOT), RDataFrame, and the local helper module `df_functions_utils.py`.

In [ ]:
import ROOT
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import kRed, kBlue
from ROOT import gStyle
import df_functions_utils as df_utils

df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/battisti/hydrogen_analysis_tests/Solid_Hydrogen2/Snap_file_processing/Processed_tree.root")
Out_dir = "Plots/SH_mu_nu_reco_performance/"

In [ ]:
### Uncoment for fast testing
df = df.Range(1000)

### Generating output file

In [ ]:
f_out = ROOT.TFile(Out_dir + "SH_reco_performance.root", "RECREATE")

# Generating df with additional columns

### List of dataframes:
- df = contains Rough_tree with all its columns
- df_extend =  df + additional columns needed in the analysis 
- df_flag = df_extended + flags that identify the filters  

### Filters flags for df_flag:
- inside the STT volume (volume_filter)
- successufull reconstruction (reco_filter)
- one track (track_filter)
- neutron space correspondence (space_filter, done with clusters) (spaceTRUE_filter, done with hit segment starts)
- neutron time correspondence (time_filter, done with clusters) (timeTRUE_filter, done with hit segment starts)
- events CC-QE with antimuon and neutron in the final state, on H (true_filter) 

In [ ]:
df_extended = df_utils.df_extend(df)
df_flag = df_utils.df_addflags(df_extended)

# Plotting

## Muon properties and reconstruction

### Muon momentum

In [ ]:
### Comment to avoid interactivity 
%jsroot

In [ ]:
### muon momentum components in MC and reco

ROOT.gStyle.SetOptStat(0)  # Disable statistics box on histograms
gDF = df_flag.Filter("volume_flag == 1") 

gH7 =  gDF.Histo1D(("h_mu_px_MC", "  ", 100, -2000, 2000),"mu_px")        #histo mu momentum MC
gH8 =  gDF.Histo1D(("h_mu_py_MC", " ", 100, -2000, 2000),"mu_py")      #histo mu momentum 
gH9 =  gDF.Histo1D(("h_mu_pz_MC", " ", 100, -10000, 20000),"mu_pz")    #histo mu momentum 

gH10 = gDF.Histo1D(("h_mu_px_reco", " ", 100, -2000, 2000), "mu_pxreco")
gH11 = gDF.Histo1D(("h_mu_py_reco", " ", 100, - 2000, 2000), "mu_pyreco")
gH12 = gDF.Histo1D(("h_mu_pz_reco", " ", 100,  -10000, 20000), "mu_pzreco")

c4 = TCanvas("c1", "Canvas", 2000, 500)
c4.Divide(3, 1)

c4.cd(1)
# gH7->SetTitle("");  // Set histogram title
gH7.GetXaxis().SetTitle("muon Momentum x (MeV)")
gH7.SetLineColor(ROOT.kBlue)
gH10.SetLineColor(ROOT.kRed)
gH10.GetXaxis().SetTitle("muon Momentum x (MeV)")
gH10.Draw()
gH7.Draw("SAME")

legend1 = TLegend(0.7, 0.75, 0.9, 0.9)   # (x1,y1,x2,y2) in coordinate NDC
legend1.AddEntry(gH7.GetPtr(), "MC", "l")
legend1.AddEntry(gH10.GetPtr(), "Reco", "l")
legend1.Draw()

c4.cd(2)
# gH8->SetTitle("");  # Set histogram title
gH8.GetXaxis().SetTitle("muon Momentum y (MeV)")
gH8.SetLineColor(ROOT.kBlue)
gH11.SetLineColor(ROOT.kRed)
gH11.GetXaxis().SetTitle("muon Momentum y (MeV)")
gH11.Draw()
gH8.Draw("SAME")

legend2 = TLegend(0.7, 0.75, 0.9, 0.9)
legend2.AddEntry(gH8.GetPtr(), "MC", "l")
legend2.AddEntry(gH11.GetPtr(), "Reco", "l")
legend2.Draw()

c4.cd(3)
# gH9->SetTitle("");  // Set histogram title
gH9.GetXaxis().SetTitle("muon Momentum z (MeV)")
gH9.SetLineColor(ROOT.kBlue)
gH12.SetLineColor(ROOT.kRed)
gH9.Draw()
gH12.Draw("SAME")

legend3 = TLegend(0.7, 0.75, 0.9, 0.9)
legend3.AddEntry(gH9.GetPtr(), "MC", "l")
legend3.AddEntry(gH12.GetPtr(), "Reco", "l")
legend3.Draw()

c4.Draw()
c4.SaveAs(Out_dir + "Mu_momentum_res_XYZ.pdf")

f_out.cd()
gH7.GetValue().Write()
gH8.GetValue().Write()
gH9.GetValue().Write()
gH10.GetValue().Write()
gH11.GetValue().Write()
gH12.GetValue().Write()

In [ ]:
### muon total momentum MC VS reco

h_P = gDF.Histo1D(("h_mu_ptot_MC", "  ", 1000, -10, 7000),"mu_P")   
h_Preco = gDF.Histo1D(("h_mu_ptot_reco", " ", 1000, -10, 7000),"mu_Preco") 
h_reco_true = gDF.Histo2D(("h_mu_ptot_recoVSMC", " ", 100, 0, 7000, 100, 0, 7000), "mu_Preco","mu_P")

ROOT.gStyle.SetOptStat(0)

c = TCanvas("c", "Canvas", 1000, 500)
c.Divide(2,1)

h_P.GetXaxis().SetTitle("muon Momentum (MeV)")
h_P.SetLineColor(ROOT.kBlue)
h_Preco.SetLineColor(ROOT.kRed)
h_Preco.GetXaxis().SetTitle("muon momentum (MeV)")
h_P.GetYaxis().SetTitle("Entries")

c.cd(1)
ROOT.gPad.SetLeftMargin(0.20)
ROOT.gPad.SetBottomMargin(0.15)
h_P.Draw()
h_Preco.Draw("SAME")

legend3 = TLegend(0.7, 0.75, 0.9, 0.9)
legend3.AddEntry(h_P.GetPtr(), "True", "l")
legend3.AddEntry(h_Preco.GetPtr(), "Reco", "l")
legend3.Draw()


c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.15)
ROOT.gPad.SetBottomMargin(0.15)
h_reco_true.GetXaxis().SetTitle("reco P#mu (MeV)")
h_reco_true.GetYaxis().SetTitle("true P#mu (MeV)")
h_reco_true.Draw("COLZ")


c.Draw()
c.SaveAs(Out_dir + "Mu_momentum_res.pdf")

f_out.cd()
h_P.GetValue().Write()
h_Preco.GetValue().Write()
h_reco_true.GetValue().Write()

In [ ]:
####Delete the dataframe to free memory
del gDF

## Muon reco performance for events with a successful reco

### Muon momentum plots

In [ ]:
### Generating the dataframe with the cut on reco_flag to be used for the resolution calculation
gDF2 = df_flag.Filter("volume_flag == 1 && reco_flag == 1")

In [ ]:
#total muon momentum 
h_P2 = gDF2.Histo1D(("h_mu_ptot_MC", " True mu mom ", 500, 0, 6000),"mu_P")   
h_Preco2 = gDF2.Histo1D(("h_mu_ptot_reco", "Reco mu mom ", 500, 0, 6000),"mu_Preco") 
h_reco_true2 = gDF2.Histo2D(("h_mu_ptot_recoVSMC", " RecovsTrue mu mom ", 100, 0, 6000, 100, 0, 6000), "mu_Preco","mu_P")

ROOT.gStyle.SetOptStat(0)

c2 = TCanvas("c2", "Canvas", 1000, 500)
c2.Divide(2,1)

h_P2.GetXaxis().SetTitle("Reconstructed muon momentum (MeV)")
h_P2.SetLineColor(ROOT.kBlue)
h_Preco2.SetLineColor(ROOT.kRed)
h_Preco2.GetXaxis().SetTitle("Reconstructed muon momentum (MeV)")
h_P2.GetYaxis().SetTitle("Entries")

c2.cd(1)
ROOT.gPad.SetLeftMargin(0.20)
ROOT.gPad.SetBottomMargin(0.15)
h_P2.Draw()
h_Preco2.Draw("SAME")

legend3 = TLegend(0.7, 0.75, 0.9, 0.9)
legend3.AddEntry(h_P2.GetPtr(), "True", "l")
legend3.AddEntry(h_Preco2.GetPtr(), "Reco", "l")
legend3.Draw()


c2.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.15)
ROOT.gPad.SetBottomMargin(0.15)
h_reco_true2.GetXaxis().SetTitle("reco P#mu (MeV)")
h_reco_true2.GetYaxis().SetTitle("true P#mu (MeV)")
h_reco_true2.Draw("COLZ")


c2.Draw()
c2.SaveAs(Out_dir + "Mu_momentum_res_goodreco_cut.pdf")

f_out.cd()
h_P2.GetValue().Write()   
h_Preco2.GetValue().Write() 
h_reco_true2.GetValue().Write()


In [ ]:
### Muon momentum components in MC and reco with reco cut

gH7 =  gDF2.Histo1D(("h_mu_px_MC_goodreco", "  ", 100, -2000, 2000),"mu_px")        #histo mu momentum MC
gH8 =  gDF2.Histo1D(("h_mu_py_MC_goodreco", " ", 100, -2000, 2000),"mu_py")      #histo mu momentum 
gH9 =  gDF2.Histo1D(("h_mu_pz_MC_goodreco", " ", 100, -10000, 20000),"mu_pz")    #histo mu momentum 

gH10 = gDF2.Histo1D(("h_mu_px_reco_goodreco", " ", 100, -2000, 2000), "mu_pxreco")
gH11 = gDF2.Histo1D(("h_mu_py_reco_goodreco", " ", 100, - 2000, 2000), "mu_pyreco")
gH12 = gDF2.Histo1D(("h_mu_pz_reco_goodreco", " ", 100,  -10000, 20000), "mu_pzreco")

c4 = TCanvas("c1", "Canvas", 1600, 500)
c4.Divide(3, 1)

c4.cd(1)
# gH7->SetTitle("");  // Set histogram title
gH7.GetXaxis().SetTitle("muon Momentum x (MeV)")
gH7.SetLineColor(ROOT.kBlue)
gH10.SetLineColor(ROOT.kRed)
gH10.GetXaxis().SetTitle("muon Momentum x (MeV)")
gH10.Draw()
gH7.Draw("SAME")

legend1 = TLegend(0.7, 0.75, 0.9, 0.9)   # (x1,y1,x2,y2) in coordinate NDC
legend1.AddEntry(gH7.GetPtr(), "MC", "l")
legend1.AddEntry(gH10.GetPtr(), "Reco", "l")
legend1.Draw()

c4.cd(2)
# gH8->SetTitle("");  # Set histogram title
gH8.GetXaxis().SetTitle("muon Momentum y (MeV)")
gH8.SetLineColor(ROOT.kBlue)
gH11.SetLineColor(ROOT.kRed)
gH11.GetXaxis().SetTitle("muon Momentum y (MeV)")
gH11.Draw()
gH8.Draw("SAME")

legend2 = TLegend(0.7, 0.75, 0.9, 0.9)
legend2.AddEntry(gH8.GetPtr(), "MC", "l")
legend2.AddEntry(gH11.GetPtr(), "Reco", "l")
legend2.Draw()

c4.cd(3)
# gH9->SetTitle("");  // Set histogram title
gH9.GetXaxis().SetTitle("muon Momentum z (MeV)")
gH9.SetLineColor(ROOT.kBlue)
gH12.SetLineColor(ROOT.kRed)
gH9.Draw()
gH12.Draw("SAME")

legend3 = TLegend(0.7, 0.75, 0.9, 0.9)
legend3.AddEntry(gH9.GetPtr(), "MC", "l")
legend3.AddEntry(gH12.GetPtr(), "Reco", "l")
legend3.Draw()

c4.Draw()
c4.SaveAs(Out_dir + "Mu_momentum_res_XYZ_goodreco_cut.pdf")

f_out.cd()
gH7.GetValue().Write()
gH8.GetValue().Write()
gH9.GetValue().Write()
gH10.GetValue().Write()
gH11.GetValue().Write()
gH12.GetValue().Write()


In [ ]:
# recoVStrue muon momentum components

# ROOT.gStyle.SetPalette(ROOT.kBird)
# ROOT.gStyle.SetUnderflowColor(ROOT.kBlue)
# ROOT.gStyle.SetFrameFillColor(ROOT.kAzure -6)
# ROOT.gStyle.SetFrameFillColor(ROOT.kBlue -3)
# ROOT.gPad.SetFrameFillColor(ROOT.TColor.GetColorPalette(0))


h2_px = gDF2.Histo2D(("h_mu_px_recoVSMC_goodreco", "RecovsTrue mu momx ",50, -1000, 1000, 50, -1000, 1000),"mu_pxreco", "mu_px")
h2_py = gDF2.Histo2D(("h_mu_py_recoVSMC_goodreco", "RecovsTrue mu momy ",50, -1000, 1000, 50, -1000, 1000),"mu_pyreco", "mu_py")
h2_pz = gDF2.Histo2D(("h_mu_pz_recoVSMC_goodreco", "RecovsTrue mu momz ",50, 0, 10000, 50, 0, 10000),"mu_pzreco", "mu_pz")

c = ROOT.TCanvas("c", "Reco vs True", 1800, 600)
# c.Divide(3, 1)
c.Divide(3, 1, 0.01, 0.05) 

# Disegna
c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetBottomMargin(0.15)
h2_px.SetMinimum(0)
h2_px.SetMaximum(1000)
h2_px.GetXaxis().SetTitle("reco px")
h2_px.GetYaxis().SetTitle("true px")
h2_px.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetBottomMargin(0.15)
h2_py.SetMinimum(0)
h2_py.SetMaximum(1000)
h2_py.GetXaxis().SetTitle("reco py")
h2_py.GetYaxis().SetTitle("true py")
h2_py.Draw()

c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetBottomMargin(0.15)
h2_pz.SetMinimum(0)
h2_pz.SetMaximum(1000)
h2_pz.GetXaxis().SetTitle("reco pz")
h2_pz.GetYaxis().SetTitle("true pz")
h2_pz.Draw()

c.Draw()
c.SaveAs(Out_dir + "Mu_momentum_res_XYZ_goodreco_cut_2D.pdf")


f_out.cd()
h2_px.GetValue().Write()
h2_py.GetValue().Write()
h2_pz.GetValue().Write()


In [ ]:
# relative momentum component residuals

c = TCanvas("c", " ", 1500, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 0.6
min = -0.6

h_resmu_x = df_flag.Histo1D(("h1_mu_px_res", " Res mu x ", 100, min, max),"res_x")
h_resmu_y = df_flag.Histo1D(("h2_mu_py_res", " Res mu y ", 100, min, max),"res_y")
h_resmu_z = df_flag.Histo1D(("h3_mu_pz_res", " Res mu z ", 100, min, max),"res_z")

c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_resmu_x.GetXaxis().SetTitle("#Delta Px/Px")
h_resmu_x.GetYaxis().SetTitle("Entries")
h_resmu_x.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_resmu_y.GetXaxis().SetTitle("#Delta Py/Py")
h_resmu_y.GetYaxis().SetTitle("Entries")

h_resmu_y.Draw()

c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_resmu_z.GetXaxis().SetTitle("#Delta Pz/Pz")
h_resmu_z.GetYaxis().SetTitle("Entries")

h_resmu_z.Draw()

c.Draw()
c.SaveAs(Out_dir + "Mu_momentum_res_XYZ_goodreco_cut_residuals.pdf")

f_out.cd()
h_resmu_x.GetValue().Write()
h_resmu_y.GetValue().Write()
h_resmu_z.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 400, 400)
ROOT.gStyle.SetOptStat(0)

max = 0.6
min = -0.6

h_res_mu_P = df_flag.Histo1D(("h_mu_P_res", " muon momentum residuals ", 100, min, max),"res")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_mu_P.GetXaxis().SetTitle("#Delta P/P")
h_res_mu_P.GetYaxis().SetTitle("Entries")
h_res_mu_P.Draw()


c.Draw()
c.SaveAs(Out_dir + "Mu_momentum_res_P_goodreco_cut_residuals.pdf")

f_out.cd()
h_res_mu_P.GetValue().Write()

In [ ]:
### muon energy residuals
c = TCanvas("c", " ", 400, 400)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 0.6
min = -0.6

h_res_mu_E = df_flag.Histo1D(("h_mu_E_res", " muon energy residuals", 100, min, max),"res_E_mu")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_mu_E.GetXaxis().SetTitle("#Delta E/E")
h_res_mu_E.GetYaxis().SetTitle("Entries")
h_res_mu_E.Draw()


c.Draw()
c.SaveAs(Out_dir + "Mu_energy_res_goodreco_cut_residuals.pdf")

f_out.cd()
h_res_mu_E.GetValue().Write()

## Neutrino quantities residuals, fully determined from muon properties + beam direction

In [ ]:
### neutrino momentum residuals for events with good topology and good reco
DF = df_flag.Filter("volume_flag == 1 && reco_flag == 1 ")

c = TCanvas("c", " ", 1500, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 0.6
min = -0.6

h_res_nu_pxS = DF.Filter('true_flag == 1').Histo1D(("h_res_nu_px_goodreco_goodtopology", " res_nu_pxS ", 100, -1.5, 0.5 ),"res_px_nu")
h_res_nu_pyS = DF.Filter('true_flag == 1').Histo1D(("h_res_nu_py_goodreco_goodtopology", " res_nu_pyS ", 100, min, max),"res_py_nu")
h_res_nu_pzS = DF.Filter('true_flag == 1').Histo1D(("h_res_nu_pz_goodreco_goodtopology", " res_nu_pzS ", 100, min, max),"res_pz_nu")

c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nu_pxS.GetXaxis().SetTitle("#Delta Px/Px")
h_res_nu_pxS.GetYaxis().SetTitle("Entries")
h_res_nu_pxS.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nu_pyS.GetXaxis().SetTitle("#Delta Py/Py")
h_res_nu_pyS.GetYaxis().SetTitle("Entries")
h_res_nu_pyS.Draw()

c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nu_pzS.GetXaxis().SetTitle("#Delta Pz/Pz")
h_res_nu_pzS.GetYaxis().SetTitle("Entries")

h_res_nu_pzS.Draw()

c.Draw()
c.SaveAs(Out_dir + "Nu_momentum_res_XYZ_goodreco_goodtopology_cut_residuals.pdf")


f_out.cd()
h_res_nu_pxS.GetValue().Write()
h_res_nu_pyS.GetValue().Write()
h_res_nu_pzS.GetValue().Write()

In [ ]:
### neutrino energy residuals
c = TCanvas("c", " ", 400, 400)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 0.6
min = -0.6

h_E_nuS = DF.Filter('true_flag == 1').Histo1D(("h_nu_E_res", " neutrino energy residuals S ", 100, min, max),"res_E_nu")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_E_nuS.GetXaxis().SetTitle("#Delta E/E")
h_E_nuS.GetYaxis().SetTitle("Entries")
h_E_nuS.Draw()


c.Draw()
c.SaveAs(Out_dir + "Nu_energy_res_goodreco_goodtopologycut_residuals.pdf")

f_out.cd()
h_E_nuS.GetValue().Write()

In [ ]:
### neutrino momentum residuals for events with good reco but bad topology
c = TCanvas("c", " ", 1500, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 1.5
min = -1.5

h_res_nu_pxB = DF.Filter('true_flag == 0').Histo1D(("h_res_nu_px_goodreco_badtopology", "res_nu_pxB  ", 100, min, max),"res_px_nu")
h_res_nu_pyB = DF.Filter('true_flag == 0').Histo1D(("h_res_nu_py_goodreco_badtopology", "res_nu_pyB  ", 100, min, max),"res_py_nu")
h_res_nu_pzB = DF.Filter('true_flag == 0').Histo1D(("h_res_nu_pz_goodreco_badtopology", " res_nu_pzB ", 100, min, max),"res_pz_nu")

c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nu_pxB.GetXaxis().SetTitle("#Delta Px/Px")
h_res_nu_pxB.GetYaxis().SetTitle("Entries")
h_res_nu_pxB.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nu_pyB.GetXaxis().SetTitle("#Delta Py/Py")
h_res_nu_pyB.GetYaxis().SetTitle("Entries")

h_res_nu_pyB.Draw()

c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_nu_pzB.GetXaxis().SetTitle("#Delta Pz/Pz")
h_res_nu_pzB.GetYaxis().SetTitle("Entries")

h_res_nu_pzB.Draw()

c.Draw()
c.SaveAs(Out_dir + "Nu_momentum_res_XYZ_goodreco_badtopology_cut_residuals.pdf")


f_out.cd()
h_res_nu_pxB.GetValue().Write()
h_res_nu_pyB.GetValue().Write()
h_res_nu_pzB.GetValue().Write()

In [ ]:
c = TCanvas("c", " ", 600, 600)
c.Divide(3,1)
ROOT.gStyle.SetOptStat(0)

max = 1.0
min = -1.0

h_res_E_nuB = DF.Filter('true_flag == 0').Histo1D(("h_res_E_nu_goodreco_badtopology", " res_E_nuB ", 100, min, max),"res_E_nu")

ROOT.gPad.SetLeftMargin(0.15)
ROOT.gPad.SetRightMargin(0.04)
h_res_E_nuB.GetXaxis().SetTitle("#Delta E/E")
h_res_E_nuB.GetYaxis().SetTitle("Entries")
h_res_E_nuB.Draw()


c.Draw()
c.SaveAs(Out_dir + "Nu_energy_res_goodreco_badtopology_cut_residuals.pdf")


f_out.cd()
h_res_E_nuB.GetValue().Write()

In [ ]:
### Deleting the dataframe to free memory
del DF

# Events with 1 good track

In [ ]:
df_one_track = df_flag.Filter("volume_flag == 1 && reco_flag == 1 && track_flag == 1")

In [ ]:
#Histo1D of the number of points 
ROOT.gStyle.SetOptStat(0)
c = TCanvas("c", " ", 600, 600)
h_point = df_one_track.Histo1D(("h_npoints_per_track_goodreco_1track", "n_point", 160, 0 , 160), "N_points")
h_point.GetXaxis().SetTitle("Number of points")
h_point.GetYaxis().SetTitle("Entries")
c.SetLogy()
h_point.Draw()
c.Draw()
c.SaveAs(Out_dir + "n_points_per_track_goodreco_1track.pdf")


f_out.cd()
h_point.GetValue().Write()


In [ ]:
#Gaussian fit

c1 = ROOT.TCanvas("c1", " ", 600, 600)

gaus = ROOT.TF1("gaus", "gaus",- 0.1, 0.1)
h = df_one_track.Histo1D(("h"," ", 100, -0.2, 0.2), "res")
h.Fit(gaus, "R")

h.GetXaxis().SetTitle("percentage residuals")
h.Draw()
gaus.Draw("SAME")


mean = gaus.GetParameter(1)
sigma = gaus.GetParameter(2)

print(f"Fit mean: {mean}, sigma: {sigma}")

legend = ROOT.TLegend(0.7, 0.75, 0.9, 0.9)
legend.AddEntry(gaus, "Gaussian fit", "l")
legend.Draw()

c1.Draw()
c1.SaveAs(Out_dir + "mu_momentum_res_gaus_fit_goodreco_1track.pdf")

In [ ]:
# Resolutions (RMS) for events with N_points = X
ROOT.gStyle.SetOptStat(0)
c9 = ROOT.TCanvas("c2", "", 1600, 600)
c11 = TCanvas("c11", "", 1000, 1000)
c11.Divide(6,6)

c9.Divide(2,1)

# 2D histogram: mu_points vs residuals
h2 = df_one_track.Histo2D(("h2", " ", 20, 0 , 150, 75, -1., 1.), "N_points", "res")
hist2 = h2.GetValue()

# Histogram to store RMS per bin
sigma_hist_RMS = ROOT.TH1D("h_mu_p_sigma_RMS_VS_npoints_goodreco_1track", " ", 
                           hist2.GetNbinsX(), 
                           hist2.GetXaxis().GetXmin(), 
                           hist2.GetXaxis().GetXmax())

# Loop over x bins
for ibin in range(1, hist2.GetNbinsX() + 1):

    # Projection on Y for single X bin
    proj = hist2.ProjectionY(f"proj_{ibin}", ibin, ibin)

    # Go to corresponding pad
    c11.cd(ibin)

    # Style
    proj.SetTitle(f"X bin {ibin}")
    proj.GetXaxis().SetTitle("#DeltaP/P")
    proj.GetYaxis().SetTitle("Entries")

    # Draw
    proj.Draw()

    # Fill RMS histogram
    if proj.GetEntries() > 0:
        rms = proj.GetRMS()
        rms_err = proj.GetRMSError()
        sigma_hist_RMS.SetBinContent(ibin, rms)
        sigma_hist_RMS.SetBinError(ibin, rms_err)

c9.cd(1)
sigma_hist_RMS.GetXaxis().SetTitle("Number of points")
sigma_hist_RMS.GetYaxis().SetTitle("Resolution from #DeltaP#mu/P#mu distribution")

sigma_hist_RMS.SetMarkerStyle(6)   # 5 = croce
sigma_hist_RMS.SetMarkerSize(1.2)  
sigma_hist_RMS.SetMarkerColor(ROOT.kBlue+1)
sigma_hist_RMS.SetLineColor(ROOT.kBlue+1)
sigma_hist_RMS.Draw("E1P")

c9.cd(2)
h = df_one_track.Histo1D(("h_mu_p_res_goodreco_1track"," ", 100, -1., 1.), "res")
h.Draw()

f_out.cd()
sigma_hist_RMS.Write()
h.GetValue().Write()

c9.Draw()
c9.SaveAs(Out_dir + "mu_p_resolutionVSpoints_goodreco_1track.pdf")
c11.Draw()
c11.SaveAs(Out_dir + "mu_p_resolutionVSpoints_perbin_goodreco_1track.pdf")

In [ ]:
# Resolution (GAUSS) for events with N_points = X
ROOT.gStyle.SetOptStat(0)
c = TCanvas()
import array

h2 = df_one_track.Histo2D(("h2", " ", 20, 0 , 150, 75, -1., 1.), "N_points", "res")
# h2 = df_extended2.Histo2D(("h2", " ", 15, 0 , 150, 100, -1., 1.), "mu_points", "res")
hist2 = h2.GetValue()


bin_edges_array = array.array('d', list(range(0, 121, 10)) + [135, 150])
sigma_hist = ROOT.TH1D("h_mu_p_sigma_gauss_VS_npoints_goodreco_1track", " ", len(bin_edges_array)-1, bin_edges_array)

#sigma_hist = ROOT.TH1D("sigma", " ",hist2.GetNbinsX(), hist2.GetXaxis().GetXmin(), hist2.GetXaxis().GetXmax())

for ibin in range(1, hist2.GetNbinsX() + 1):
    proj = hist2.ProjectionY(f"proj_{ibin}", ibin, ibin)
    
    if proj.GetEntries() < 1:
         continue
    
    fit = proj.Fit("gaus", "QS")  #Store result senza stamparlo

#if proj.GetEntries() > 0:
    #fit = proj.Fit("gaus", "QS")  #Store result senza stamparlo
    if int(fit) == 0:  # fit riuscito
            sigma = fit.Parameter(2)
            sigma_err = fit.ParError(2)
            sigma_hist.SetBinContent(ibin, sigma)
            sigma_hist.SetBinError(ibin, sigma_err)
#sigma_hist.SetMarkerStyle(6) #20
sigma_hist.GetXaxis().SetTitle("Number of points")
# sigma_hist.GetXaxis().SetTitle("number of points")
# sigma_hist.GetXaxis().SetRangeUser(0, 140)
sigma_hist.GetYaxis().SetTitle("Resolution from #DeltaP#mu/P#mu distribution")

sigma_hist.SetMarkerStyle(6)   # 5 = croce
# sigma_hist.SetMarkerSize(1.2)  # dimensione della croce
sigma_hist.SetMarkerColor(ROOT.kBlue+1)
sigma_hist.SetLineColor(ROOT.kBlue+1)

sigma_hist.Draw("E1P")
c.Draw()
c.SaveAs(Out_dir + "mu_p_resolutionVSpoints_gaus_goodreco_1track.pdf")

f_out.cd()
sigma_hist.Write()


In [ ]:
# Resolution (GAUSS) for events vs momentum with N_points = X
ROOT.gStyle.SetOptStat(0)
c = TCanvas("c", " ", 600, 600)

h2 = df_one_track.Histo2D(("h_", " ", 20, 0 , 6000, 100, -1., 1.), "mu_P", "res")
# h2 = df_extended2.Histo2D(("h2", " ", 15, 0 , 150, 100, -1., 1.), "mu_points", "res")
hist2 = h2.GetValue()

sigma_hist2 = ROOT.TH1D("mu_p_sigma_gauss_VS_p_goodreco_1track", " ",hist2.GetNbinsX(), hist2.GetXaxis().GetXmin(), hist2.GetXaxis().GetXmax())

for ibin in range(1, hist2.GetNbinsX() + 1):
    proj = hist2.ProjectionY(f"proj_{ibin}", ibin, ibin)
    
    if proj.GetEntries() < 1:
         continue
    
    fit = proj.Fit("gaus", "QS")  #Store result senza stamparlo

#if proj.GetEntries() > 0:
    #fit = proj.Fit("gaus", "QS")  #Store result senza stamparlo
    if int(fit) == 0:  # fit riuscito
            sigma2 = fit.Parameter(2)
            sigma_err2 = fit.ParError(2)
            sigma_hist2.SetBinContent(ibin, sigma)
            sigma_hist2.SetBinError(ibin, sigma_err)
#sigma_hist.SetMarkerStyle(6) #20
sigma_hist2.GetXaxis().SetTitle("muon momentum (Mev)")
# sigma_hist.GetXaxis().SetTitle("number of points")
# sigma_hist.GetXaxis().SetRangeUser(0, 140)
sigma_hist2.GetYaxis().SetTitle("Resolution from #DeltaP#mu/P#mu distribution")

sigma_hist2.SetMarkerStyle(6)   # 5 = croce
# sigma_hist.SetMarkerSize(1.2)  # dimensione della croce
sigma_hist2.SetMarkerColor(ROOT.kBlue+1)
sigma_hist2.SetLineColor(ROOT.kBlue+1)

sigma_hist2.Draw("E1P")
c.Draw()
c.SaveAs(Out_dir + "mu_p_resolutionVSp_gaus_goodreco_1track.pdf")

f_out.cd()
sigma_hist2.Write()


## Close output file

In [ ]:
f_out.Close()